In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
# category_encoders, feature_tools, imblearn, tsfresh

/kaggle/input/criteo-dataset/dac/test.txt
/kaggle/input/criteo-dataset/dac/train.txt
/kaggle/input/criteo-dataset/dac/readme.txt


In [2]:
train_f_name = '/kaggle/input/criteo-dataset/dac/train.txt'
test_f_name = '/kaggle/input/criteo-dataset/dac/test.txt'

I_col_names = ['I{}'.format(i) for i in range(1, 14)]
C_col_names = ['C{}'.format(i) for i in range(1, 27)]

raw_tr_rows_cnt = 45840617 # rows in training set
raw_ts_rows_cnt = 6042135  # rows in testing set

In [3]:
from tqdm import tqdm_notebook
import collections
import gc
counts = [collections.defaultdict(int) for _ in range(26)]

'''
Comment by Runtong:
    I once tried to load source file as csv format but failed. The training set is toooooo large 
and pandas or csv module couldn't handle it(pd.read_csv or csv.DictReader). So, I used the most 
basic method to open the file and do parts of feature engineering based on that.   
'''

f = open(train_f_name,'r')
i = 0

ff = f.readlines()


In [4]:
for i,line in tqdm_notebook(enumerate(ff[:raw_tr_rows_cnt//2])):
    if i % 10 != 0: # Runtong: do downsampling, for time consumption concideration.
        continue
    row = line.strip('\n').split('\t')
    label = row[0]
    for j in range(1, 27):
        #field = 'C{0}'.format(j)
        #print(row[j - 27])
        if row[j - 27]:
            value = np.int32(int(row[j - 27],16))
            j -= 1
            counts[j][value] += 1
    del line
gc.collect()

f.close()

In [5]:
del ff
gc.collect()

0

In [6]:
freq_items = [collections.defaultdict(lambda : -1) for _ in range(26)]
Freq_Threshold = 5
# filter
for ind, dic in enumerate(counts[:26]):
    for k, c in dic.items():
        if c >= Freq_Threshold:
            freq_items[ind][k] = c
# counter
for ind, dic in enumerate(freq_items[:26]):
    for i, k in enumerate(dic.keys()):
        freq_items[ind][k] = i


In [7]:
list(zip(map(len,freq_items), map(len,counts)))

[(833, 1416),
 (530, 545),
 (25847, 738722),
 (27047, 252099),
 (207, 297),
 (14, 16),
 (10004, 11558),
 (408, 620),
 (3, 3),
 (17413, 42244),
 (4455, 5032),
 (26118, 648561),
 (3068, 3126),
 (26, 26),
 (7065, 10738),
 (26762, 481906),
 (10, 10),
 (3286, 4444),
 (1616, 1965),
 (3, 3),
 (26509, 579057),
 (11, 15),
 (15, 15),
 (16477, 67996),
 (60, 84),
 (13391, 47429)]

In [8]:
train_pec = 0.1

for pec in tqdm_notebook(range(5)):
    # each time use 10% of the dataset. start from 50%
    
    train_pd = pd.read_csv(train_f_name,
                      sep = '\t',
                      names = ['label'] + I_col_names + C_col_names,
                      nrows = int(train_pec * raw_tr_rows_cnt),
                      dtype = dict(zip(I_col_names, [np.float32] * len(I_col_names))),
                      converters = dict(zip(C_col_names, [lambda x: int(x or 'ffffffff', 16)  for _ in C_col_names])),
                      skiprows = int((0.5 + pec * 0.1)* raw_tr_rows_cnt)
                      )
    result = None
    train_pd[C_col_names] = train_pd[C_col_names].astype(np.int32)

    for i in range(1, 27):
        #print(i, 'train')
        train_pd['C{}'.format(i)] = train_pd['C{}'.format(i)].map(lambda x: freq_items[i-1][x]).astype(np.int32)
        #print(i, 'test')    
    train_pd.to_csv('./train_{}_{}.csv'.format(50 + pec * 10, 60 + pec * 10),index=False,header=False)